# Optimisation – Project 1

## Part 3.2
Median problem with weighted Euclidean distance

\begin{equation}
\min_{x \in \mathbb{R}} \sum_{i \in \mathcal{M}} v^i d_2(a^i, x)
\end{equation}

where $\mathcal{M} = \{1, \dots, m\}$ and $0 ≤ v^i \in \mathbb{R}, \; \forall i \in \mathcal{M}$.

We will here look into the Weiszfeld algorithm for solving the problem above.

## See overleaf for some of the solutions

## 4

In [4]:
import autograd.numpy as np
from autograd import grad
import time

## A solution for implementing an apropriate stopping criterion

In [15]:
A = np.random.random(size = (2, 10))
x = np.array([0.5, 0.5])
v = np.random.rand(10)
v = v/np.sum(v)
np.sum(v), v

(1.0000000000000002,
 array([0.08833373, 0.11989191, 0.11138936, 0.12770299, 0.06075476,
        0.13134149, 0.16821206, 0.09414424, 0.07888017, 0.01934929]))

In [22]:
def ourWeiszfeld(A, v, epsilon = 10**(-7)):
    """
    A = [a^1,..., a^i,..., a^m]^T   where   a^i = [a_1^i, a_2^i]
    Hence, A is a m times 2 matrix
    v = [v^1,..., v^i,..., v^m]^T   where   0<v^i \in \mathbb{R}, \forall i \in \mathcal{M} 
    epsilon > 0
    """
    m = A.shape[1]
    
    #############################
    # Test if some positions are optimal
    #############################
    
    # We define a test function to make the code leaner
    def testK(k):
        sum1 = 0
        sum2 = 0
        for i in range(m): 
            if i != k:
                dist = np.linalg.norm((A[:, k] - A[:, i])) # d_2(*,*) euclidean distance
                sum1 += v[i]* (A[:, k].item(0) - A[:, i].item(0))/(dist)
                sum2 += v[i]* (A[:, k].item(1) - A[:, i].item(1))/(dist)
                
        result = np.sqrt(sum1**2 + sum2**2)
        return (result <= v[k])

    
    for k in range(m): 
        # theorem 1 test if the theorem is fulfilled for a "k"
        if testK(k): # do we need to pass the k? should rather not?
            x_star = A[:, k] # could just return A[:, k], but this is clearer
            return(x_star)
    
    # choose a starting point x = [x_1, x_2]^T, can be found solving the median problem with ||\cdot||_2^2
    x = np.array([0.463, 0.296])
    
    
    #############################
    # Functions for stopping ciretion
    #############################
    
    def sigma(x):
        dist = np.linalg.norm(x[:, None] - A, axis = 0)
        return np.max(dist)
    
    def f_d2(x):
        return (np.sum(np.multiply(v,np.linalg.norm(x[:, None] - A, axis = 0))))

    gradient = grad(f_d2)
    
    def stoping_criterion(x):
        norm_grad = np.linalg.norm(gradient(x))
        sig = sigma(x)
        return ((norm_grad * sig)/(f_d2(x) - norm_grad * sig))
    
    k = 0
    while stoping_criterion(x) > epsilon and k < 50: # We have k = 50 here just to make the while loop stop
        k += 1
        # We print the value of the stopping criterion to see what happens
        print(stoping_criterion(x))
        
        for j in range(len(x)):
            enum = 0
            deno = 0
            
            for i in range(m):
                dist = np.linalg.norm((np.squeeze(np.array(A[:, i])) - x))
                enum += v[i]*A[:, i].item(j)/dist
                deno += v[i]/dist

            x[j] = enum/deno
        
    return x

## Gradient descent algorithm

In [17]:
def steepestDescent(x_init, obj_func, max_iter = 10000, threshold = 10**(-7)):
    
    # We create the gradient function for the objective function using autograd
    gradient = grad(obj_func)
    
    # Here we initialize
    i = 0
    x = np.array(x_init) 
    rho = 0.4
    c = 0.6
    diff = np.full((len(x), 1), 100) # initialize some value
    while i < max_iter and diff.any() > threshold: # Can use same stopping criterion as above to measure time performance
        i += 1
        a = 1
        grad_k = gradient(x)
        # I make a pk for convenience 
        pk = -grad_k
        
        # The following is backtracking
        while obj_func(x + a*pk) > obj_func(x) + a*c*np.transpose(grad_k) @ pk: # i.e. repeat until not true
            a = rho*a
        
        # The new x is stored
        x = x + a*pk
        diff = abs(a*pk)
    return x

In [18]:
def sigma(x):
    dist = np.linalg.norm(x[:, None] - A, axis = 0)
    return np.max(dist)
    
def f_d2(x):
    return (np.sum(np.multiply(v,np.linalg.norm(x[:, None] - A, axis = 0))))

gradient = grad(f_d2)

def stoping_criterion(x):
    norm_grad = np.linalg.norm(gradient(x))
    sig = sigma(x)
    return ((norm_grad * sig)/(f_d2(x) - norm_grad * sig))

In [19]:
steepest_descent_result = steepestDescent(x, f_d2)

In [20]:
stoping_criterion(steepest_descent_result), steepest_descent_result

(1.0717655952351659e-11, array([0.59897712, 0.58037236]))

In [23]:
ourWeiszfeld(A, v), steepest_descent_result, A

10.413440381177496
0.6291569517484836
0.2382598955049261
0.11843077136833274
0.06657394528706945
0.039458161858154546
0.023752722588723036
0.01435648013790805
0.008688274162302298
0.0052611927904166545
0.0031871360303251622
0.0019311747565478035
0.001170316163059281
0.0007092772368686116
0.00042987507572873955
0.00026053874472018644
0.00015790707633500564
9.57036101051107e-05
5.800322513949439e-05
3.515388471324858e-05
2.130553002218571e-05
1.2912480819392144e-05
7.825746788465751e-06
4.7428674897832015e-06
2.874455223855166e-06
1.7420862521109807e-06
1.0558043466222014e-06
6.398777689884668e-07
3.8780236196362444e-07
2.350302583394002e-07
1.4244166384505928e-07


(array([0.59897712, 0.58037234]),
 array([0.59897712, 0.58037236]),
 array([[0.37645758, 0.34418904, 0.45974131, 0.82427527, 0.8950342 ,
         0.35760366, 0.65227289, 0.88005478, 0.74367886, 0.36362856],
        [0.1395243 , 0.42060374, 0.74059467, 0.31761286, 0.82686015,
         0.81317752, 0.67095339, 0.09289106, 0.44203628, 0.96070041]]))

### From the what we see here, the stopping criterion seem to work as well as the steepest descent algorithm

In [259]:
A1 = np.array([[1, 0, 0], [0, 0, 1]])
v1 = np.array([0.25, 0.5, 0.75])
A1, v1


(array([[1, 0, 0],
        [0, 0, 1]]), array([0.25, 0.5 , 0.75]))

In [25]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull, convex_hull_plot_2d   # get a convex hull computation algorithm
 
 
 
#points = np.random.rand(10, 2)   # 30 random points in 2-D
#hull = ConvexHull(points)
 
#vecs = []
#for i in range(len(hull.vertices)):
 ##   element = points[hull.vertices[i]]
 #   element = element.tolist()
 #   vecs.append(element)
 
def distance(x_1,x_2):
    x_1 = np.array(x_1)
    x_2 = np.array(x_2)
    diff = x_1 - x_2
    return np.linalg.norm(diff, 2) 
 
 
def isCounterClockwise(n1,n2, n3):
    return ((n1 <n2 <n3) or (n3 < n1 < n2) or (n2 < n3 < n1))
 
def Area(vtxList, n1, n2, n3): # get area of triangle made of point 1,2, 3 from a list of vertices
    n = len(vtxList)
    vtx1 = vtxList[n1]
    vtx2 = vtxList[n2]
    vtx3 = vtxList[n3]
    # print(vtx1, vtx2, vtx3)
    a = distance(vtx1,vtx2)
    b = distance(vtx3,vtx2)
    c = distance(vtx1,vtx3)
    s = (a + b + c) / 2
    area = (s*(s-a)*(s-b)*(s-c)) ** 0.5
    if isCounterClockwise(n1,n2,n3):
        return round(area,3)
    else:
        return round(-area,3)
 
 
def GetAllAntiPodalPairs(vtxList):
    pairList = []
    n = len(vtxList)
    i_1 = 1
    i_n = 0
    i = i_n
    j = (i + 1)%n
    while (Area(vtxList,i, (i + 1)%n, (j + 1)%n) > Area(vtxList, i, (i + 1)%n, j)):
        j = (j + 1)%n
        j0 = j
    while (j != i_1):
        i = (i + 1)%n
        pairList.append([i, j])
        while ((Area(vtxList, i, (i + 1)%n, (j + 1)%n) > Area(vtxList, i, (i + 1)%n, j))):
            j = (j + 1)%n
            if (j != i_1):
                pairList.append([i, j])
        if (Area(vtxList, i, (i + 1)%n, (j + 1)%n) == Area(vtxList, i, (i + 1)%n, j)):
            if ((i, j) != (j0, i_n)):
                pairList.append([i, (j + 1)%n])
 
    return(pairList)
 
 
 
def findDiameter(polygon):
    maxx = 0
    antiPodalPairs = GetAllAntiPodalPairs(polygon)
    for pair in antiPodalPairs:
        pairPoint1 = polygon[pair[0]]
        pairPoint2 = polygon[pair[1]]
        candidate = distance(pairPoint1, pairPoint2)
        if (candidate > maxx):
            maxx = candidate
    return maxx
 
 
# EXAMPLE
 
polygonZ = [[0,0], [3,0], [2,1], [0,1]]
print(findDiameter(polygonZ))

3.1622776601683795


## 5

Compare using precision, speed and possibly stability
